In [457]:
import pandas as pd
import numpy as np
import pymysql
import sqlalchemy as db


CARGAR DATASETS

In [458]:
producto = pd.read_parquet('Datasets/producto.parquet')

In [459]:
sucursal = pd.read_csv('Datasets/sucursal.csv')

In [460]:
precio_1 = pd.read_csv('Datasets/precios_semana_20200413.csv', encoding='UTF-16 LE')

precio_2 = pd.read_excel('Datasets/precios_semanas_20200419_20200426.xlsx')

precio_3 = pd.read_json('Datasets/precios_semana_20200503.json')



DEFINIR TIPO DE DATOS EN LAS COLUMNAS DE LOS DATAFRAMES CARGADOS: 

precio -> float64

producto_id ->str

sucursal_id -> str

In [461]:
def quitar_guion(codigo):  # esta es una funcion para quitar los '-' de las columnas producto_id
    y= codigo.replace("-","")
    return y


In [462]:
data_types_dict = {'producto_id':str , 'sucursal_id':str}
precio_1 = precio_1.astype(data_types_dict) # convierto columnas a string para usar el replace() de la funcion quitar_guion()
precio_3 = precio_3.astype(data_types_dict)

precio_1['producto_id']= precio_1['producto_id'].map(quitar_guion) # quito guiones de ids de producto
precio_3['producto_id']= precio_3['producto_id'].map(quitar_guion)
producto['id']= producto['id'].map(quitar_guion)

precio_1['producto_id'] = precio_1['producto_id'].apply(lambda x : np.NaN if x=='nan' else x) # asigno NAN a campos precio vacios
precio_3['precio'] = precio_3['precio'].apply(lambda x : None if x=='' else x) # asigno NAN a campos precio vacios


In [463]:
data_types_dict = {'producto_id':float , 'precio':float} # cambio tipo a float para poder trabajar con NAN
precio_1 = precio_1.astype(data_types_dict)  
precio_3 = precio_3.astype(data_types_dict)
producto = producto.astype({'id':float}) # esta conversion se realiza debido a que no es posible convertir directamente a int
producto = producto.astype({'id':np.int64})


UNIFICAR ORDEN DE COLUMNAS

In [464]:
precio_2 = precio_2.reindex(columns=['precio','producto_id','sucursal_id'])

UNIR DATAFRAMES DE PRECIOS

In [465]:
precio = pd.concat([precio_1 , precio_2 , precio_3])
precio.reset_index(inplace=True, drop=True)

ELIMINAR NULOS

In [466]:
precio['sucursal_id'] = precio['sucursal_id'].apply(lambda x : np.NaN if x=='nan' else x) # convierte 'nan' de columna sucursal_id en np.NAN
# se hace despues de unir ya que el tipo de sucursal_id es str en las tres tablas

In [467]:
precio.dropna(subset=['precio','sucursal_id'], inplace=True) # elimina los que no tienen precio o sucursal_id

In [468]:
precio.isnull().value_counts() # aqui se puede ver las instancias con nulos en producto_id únicamente

precio  producto_id  sucursal_id
False   False        False          1331514
        True         False            13420
dtype: int64

In [469]:
producto.drop(columns=['categoria1' , 'categoria2' , 'categoria3'], inplace=True) # eliminar las 3 col de categoria porque son en su mayoria nulos
# en la tabla sucursal no hay nulos 

ELIMINACION DE DUPLICADOS

In [470]:
#precio[precio.duplicated()] # verificar existencia de duplicados
precio.drop_duplicates(inplace=True)

DEFINIR NOMBRE DE COLUMNAS

In [471]:
producto.columns=['id_producto','marca','nombre', 'presentacion']
sucursal.columns=['id_sucursal' , 'id_comercio', 'id_bandera', 'bandera', 'razon_social', 'provincia', 'localidad', 'direccion', 'latitud', 'longitud', 'nombre', 'tipo']
precio.columns=['precio', 'id_producto', 'id_sucursal']

ASIGNAR None A LOS id_sucursal DE LA TABLA PRECIOS QUE NO ESTÁN EN LA TABLA DE SUCURSALES 

In [472]:
merge_ps=pd.merge(precio , sucursal , how='left' , on='id_sucursal')
merge_ps.loc[:,'id_sucursal'][merge_ps.id_comercio.isnull()] = None
precio = merge_ps.loc[:,['precio','id_producto','id_sucursal']]

C:\Users\jcgut\AppData\Local\Temp\ipykernel_9728\1168758963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_ps.loc[:,'id_sucursal'][merge_ps.id_comercio.isnull()] = None


CARGA INICIAL DE TABLAS A LA BD

In [479]:
#establecimiento de conexión con la base de datos pi_1
database_username='root' # Nombre de cliente en MySQL Workbench
database_password='4488' # Contraseña de MySQL Workbench
database_ip='localhost' # localizacion del servidor
database_name='pi_1' # Nombre de Base de datos a la que nos conectaremos
engine=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
conexion=engine.connect()
#metadata=db.MetaData() # !!!!!!!!!!!!!!!pendiente!!!!!!!!!!!!!!!!

In [475]:
# cada carga tiene una definicion previa de un diccionario con los tipos con los que se desea ingestar la data en la base de datos

types_pro_dict={'id_producto':db.types.BIGINT, 
                'marca':db.types.NVARCHAR(length=40), 
                'nombre':db.types.NVARCHAR(length=200), 
                'presentacion':db.types.NVARCHAR(length=20)
                }
producto.to_sql('productos',conexion, index=False , dtype=types_pro_dict) # carga de la tabla producto

types_suc_dict={'id_sucursal':db.types.VARCHAR(length=10), 
                'id_comercio':db.types.INTEGER, 
                'id_bandera':db.types.INTEGER, 
                'bandera':db.types.VARCHAR(length=80),
                'razon_social':db.types.VARCHAR(length=100), 
                'provincia':db.types.VARCHAR(length=10), 
                'localidad':db.types.VARCHAR(length=80), 
                'direccion':db.types.VARCHAR(length=80), 
                'latitud':db.types.DECIMAL(precision=8, scale=6), 
                'longitud':db.types.DECIMAL(precision=8, scale=6), 
                'nombre':db.types.VARCHAR(length=80), 
                'tipo':db.types.VARCHAR(length=50)
                }
sucursal.to_sql('sucursales',conexion, index=False , dtype = types_suc_dict ) # carga de la tabla sucursal

types_precio_dict={'precio':db.types.FLOAT, 'id_producto':db.types.BIGINT, 'id_sucursal':db.types.VARCHAR(length=10)}
precio.to_sql('precios',conexion, index=False, dtype=types_precio_dict) # carga de la tabla precio

1168551

In [478]:
conexion.close()